In [1]:
import os
from os import path
import sqlite3
global_sqlite_filename='apf.db'
import pandas
from datetime import datetime, timedelta
import numpy as np
from astropy.time import Time
from sqlalchemy import create_engine
import pytz
import numpy as np

In [2]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

#print(os.getcwd())
#print(os.listdir())

TEXT


In [3]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def get_velocity_columns():
    columns = [('MdptJulian', ct.REAL),
               ('velocity', ct.REAL),
               ('velocity_error', ct.REAL),
               ('stellar_activity1', ct.REAL),
               ('stellar_activity2', ct.REAL),
               ('total_counts', ct.REAL),
               ('brad_doesnt_know', ct.REAL),
               ('DateTimeUTC', ct.TEXT),
               ('DateTimeLocal', ct.TEXT),
               ('starID', ct.TEXT)
              ]
    return columns
    
def create_telemetry_table(cursor):        
    columns = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
    cursor.execute(sql_create_string)

    return


def create_velocity_table(cursor):
    columns = get_velocity_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns]
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE velocity (' + full_list + ', PRIMARY KEY (DateTimeLocal))'
    print(sql_create_string)  
    cursor.execute(sql_create_string)

    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [124]:
conn = open_connection()
cursor = conn.cursor()
#create_telemetry_table(cursor)
#create_velocity_table(cursor)

CREATE TABLE velocity (MdptJulian REAL, velocity REAL, velocity_error REAL, stellar_activity1 REAL, stellar_activity2 REAL, total_counts REAL, brad_doesnt_know REAL, DateTimeUTC TEXT, DateTimeLocal TEXT, starID TEXT, PRIMARY KEY (DateTimeLocal))


In [5]:
def populate_telemetry_table(conn, filenames):
    for csv_file in filenames:
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        
    


        

In [ ]:
#populate_telemetry(conn, '../telemetry_data', batchsize=10)

In [125]:
def load_velocity_csv(dirname, filename):
    full_path = os.path.join(dirname, filename)
    columns = get_velocity_columns()[:-2] # leave off the last two column names
                                        # because we're calculating them below.
    df = pandas.read_csv(full_path, sep='\s+',
            names=[x[0] for x in columns]) 
                                                
    return df
    
    
california_tzinfo = pytz.timezone('US/Pacific')

# returns a string for a single julian date. Couldn't figure out how to vectorize.
def convert_julian_to_datetime_local(juliantime):
    times = Time(juliantime, format='jd')
    full_iso = times.to_datetime(timezone=california_tzinfo).isoformat()
    return full_iso[:-6]

# takes in an array of julian dates. Returns an array of local isot strings.
def make_array_of_local_datetimes(julian_array):
    output_array = output_array = np.chararray(len(julian_array), itemsize=26, unicode=True)
    for i, julian_time in enumerate(julian_array):
        output_array[i] = convert_julian_to_datetime_local(julian_time)
    return output_array
    

def convert_julian_to_datetime_utc(juliantimes):
    times = Time(juliantimes, format='jd')
    return times.isot 
    
def add_calculated_columns(df, filename):
    new_df = df.copy()
    #datetimes = convert_julian_to_datetime(new_df.MdptJulian)
    new_df['DateTimeUTC'] = convert_julian_to_datetime_utc(new_df.MdptJulian)
    new_df['DateTimeLocal'] = make_array_of_local_datetimes(new_df.MdptJulian)
    new_df['starID'] = [filename] * len(new_df.index) 
    
    return new_df
    

def populate_velocity(conn, dirname='../standard_star_velocity_measurements/', debug=False):
    filenames = ['HD10700_APF.vels', 'HD185144_APF.vels', 'HD9407_APF.vels']
    for file in filenames:
        df_incomplete = load_velocity_csv(dirname, file)
        df_complete = add_calculated_columns(df_incomplete, file)
        if debug:
            print(df_complete)
        else: 
            df_complete.to_sql('velocity', conn, if_exists='append', index=False)
        print('tried to add {} entries to \'velocity\''.format(len(df_complete)))

#velocity_df = load_velocity_csv('../standard_star_velocity_measurements/', 'HD10700_APF.vels')
#add_calculated_columns(velocity_df, 'HD10700_APF.vels' )


In [126]:
populate_velocity(conn, debug=False)

tried to add 755 entries to 'velocity'
tried to add 1655 entries to 'velocity'
tried to add 694 entries to 'velocity'


In [8]:
# create additional calculated columns on velocity
engine = create_engine('sqlite:///apf.db', echo=False)


In [128]:
df = pandas.read_sql_query("""select DateTimeLocal from velocity limit 10
                        """, engine)
print(df)

                DateTimeLocal
0  2013-06-28T00:20:02.111994
1  2013-06-28T00:21:49.248005
2  2013-06-28T00:23:36.383976
3  2013-06-28T03:45:06.336021
4  2013-07-05T22:47:05.856020
5  2013-07-05T22:48:59.904003
6  2013-07-05T22:50:54.816033
7  2013-07-05T22:52:49.728023
8  2013-07-05T22:54:44.640013
9  2013-07-05T22:56:39.552003


In [28]:
#df = pandas.read_sql_query("""select distinct event from telemetry
                        
#                        limit 100
#                        """, engine)

df

,EVENT
0,ControllerReady
1,EraseBegin
2,ExposureBegin
3,ReadoutBegin
4,ReadoutEnd
5,ExposureEnd
6,AbortRequested
7,ExposureAborted
8,ReadoutAbort
9,CriticalMessageCamr


In [41]:
df = pandas.read_sql_query("""select distinct event from 
                        (select event from telemetry where 
                        DateTime between "2015-01-01T17:00:00.0000" and 
                        "2015-01-02T19:01:00.0000") 
                   
                        
                        """, engine)

df

,event
0,ControllerReady
1,EraseBegin
2,ExposureBegin
3,ReadoutBegin
4,ReadoutEnd
5,ExposureEnd


In [82]:
df = pandas.read_sql_query(
"""SELECT
  event,
  COUNT(*) AS `num`
FROM
  (select * from telemetry where 
                        DateTime between "2015-02-02T17:00:00.0000" and 
                        "2015-02-03T05:01:00.0000") 
GROUP BY
  event                  
                        """, engine)

df


,EVENT,num
0,ControllerReady,20647
1,EraseBegin,43
2,ExposureBegin,10290
3,ExposureEnd,10
4,ReadoutBegin,812
5,ReadoutEnd,11


In [60]:
df = pandas.read_sql_query(
"""SELECT DateTime, event from (select * from telemetry where DateTime < "2015-01-02T19:01:00.0000"
    and (event == "ExposureEnd" or event == 'ReadoutEnd')) order by datetime
limit 100
                        """, engine)

df


,DateTime,EVENT
0,2015-01-02T17:54:02.1588,ReadoutEnd
1,2015-01-02T17:57:25.1011,ExposureEnd
2,2015-01-02T18:01:58.2331,ExposureEnd
3,2015-01-02T18:05:05.9387,ExposureEnd
4,2015-01-02T18:05:47.2942,ReadoutEnd
5,2015-01-02T18:09:41.7525,ExposureEnd
6,2015-01-02T18:19:14.7921,ReadoutEnd
7,2015-01-02T18:21:27.1005,ReadoutEnd
8,2015-01-02T18:40:34.7239,ExposureEnd
9,2015-01-02T18:41:16.0776,ReadoutEnd


In [122]:
df1 = pandas.read_sql_query("""
    Select cast(Datetimelocal as text) from velocity 
    where cast(DateTimeLocal as text) > "2015-04-01T17:01:00.0000"
    and cast(DateTimeLocal as text) < "2015-04-10T17:01:00.0000"
    order by DateTimeLocal limit 100

    """, engine)
df2 = pandas.read_sql_query(
"""SELECT datetime, midptfin, event from telemetry where DateTime between "2015-04-02T01:30:00.0000" 
and "2015-04-02T01:40:29.0000" and event != "ExposureBegin"
                        """, engine)

print(df1)
with pandas.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df2)


  cast(Datetimelocal as text)
0  2015-04-02T01:31:44.832007
1  2015-04-02T01:34:33.312002
2  2015-04-02T01:37:19.200018
3  2015-04-02T01:39:53.856029
4  2015-04-05T01:38:42.144006
5  2015-04-10T00:44:24.863987
6  2015-04-10T00:45:58.176014
7  2015-04-10T00:47:34.943986
                     DateTime                    MIDPTFIN            EVENT
0    2015-04-02T01:30:00.0330  2015-04-02T08:21:48.273822  ControllerReady
1    2015-04-02T01:30:01.0346  2015-04-02T08:21:48.273822  ControllerReady
2    2015-04-02T01:30:02.0391  2015-04-02T08:21:48.273822  ControllerReady
3    2015-04-02T01:30:03.0401  2015-04-02T08:21:48.273822  ControllerReady
4    2015-04-02T01:30:04.0416  2015-04-02T08:21:48.273822  ControllerReady
5    2015-04-02T01:30:05.0477  2015-04-02T08:21:48.273822  ControllerReady
6    2015-04-02T01:30:07.0224  2015-04-02T08:21:48.273822  ControllerReady
7    2015-04-02T01:30:08.0362  2015-04-02T08:21:48.273822  ControllerReady
8    2015-04-02T01:30:09.0405  2015-04-02T08:21:48.2738